In [24]:
import pandas as pd
import numpy as np
import random

In [ ]:
groups = ['Queen', 'Led Zeppelin', 'Black Sabbath', 'Deep Purple', 'Scorpions',
          'Metallica','Iron Maiden',  'Megadeth', 'Judas Priest', 'Motorhead',
          'Kendrick Lamar', 'Tyler, the Creator', 'Kanye West', '2pac', 'Jay-Z',
          'Radiohead', 'Bjork', 'My Bloody Valentine', 'Madvillain', 'Godspeed You Black Emperor!',
          'Michael Jackson', 'Madonna', 'George Michael', 'Janet Jackson', 'Prince']

In [ ]:
groups = groups[:5] + groups[10:15]
group_df = pd.DataFrame(index=groups, columns=groups)
popularities = []
for i in range(len(groups) // 5):
    popularities.extend(range(5, 0, -1))


for i in range(len(groups) // 5):
    for j in range(5 * i):
        for k in range(5 * i, 5 * i + 5):
            group_df.iloc[j, k] = group_df.iloc[k, j] = 1
    for j in range(5 * i, 5 * i + 5):
        for k in range(5 * i, 5 * i + 5):
            if j != k:
                group_df.iloc[j, k] = group_df.iloc[k, j] = 3
            else:
                group_df.iloc[j, k] = 0
    for j in range(5 * i + 5, len(groups)):
        for k in range(5 * i, 5 * i + 5):
            group_df.iloc[j, k] = group_df.iloc[k, j] = 3

for i in range(len(groups)):
    group_df.iloc[i] *= popularities[i]
group_df

In [ ]:
transaction_count = 50_000
transactions = {'Transaction': [], 'Item': []}
for tr_id in range(transaction_count):
    items = random.choices(groups, weights=popularities)
    item_count = random.randint(1, 4)
    new_items = set(random.choices(groups, weights=group_df[items[0]], k=item_count - 1)).difference(set(items))
    items.extend(list(new_items))
    for item in items:
        transactions['Transaction'].append(tr_id + 1)
        transactions['Item'].append(item)
transactions = pd.DataFrame(transactions)
transactions

In [ ]:
transactions.to_csv('groups-long.csv', index=False, encoding='utf-8')

# Анализ датасета

In [2]:
import pandas as pd
from collections import defaultdict
from tqdm import tqdm

In [45]:
df = pd.read_csv("./groups-long.csv", encoding='utf-8')

In [46]:
df.head(100)

,Transaction,Item
0,1,Deep Purple
1,1,Black Sabbath
2,1,Led Zeppelin
3,2,Kendrick Lamar
4,2,Kanye West
...,...,...
95,37,Led Zeppelin
96,37,Deep Purple
97,38,Led Zeppelin
98,38,Queen


In [47]:
df['Item'] = df['Item'].apply(lambda x: x.lower().strip())
df['Item'] = df.loc[df['Item'] != 'none', ['Item']]
df = df.dropna()

In [48]:
len(df['Item'].unique())

10

In [49]:
df['Transaction'].max() == len(df['Transaction'].unique())

np.True_

In [50]:
unique_items = df['Item'].unique()
_dict = defaultdict(list)
for transaction_number in tqdm(df['Transaction'].unique()):
    products = df.loc[df['Transaction'] == transaction_number]['Item'].to_list()
    for pr in unique_items:
        if pr in products:
            _dict[pr].append(1)
        else:
            _dict[pr].append(0)
norm_df = pd.DataFrame(_dict)
norm_df

100%|██████████| 50000/50000 [00:14<00:00, 3368.74it/s]


,deep purple,black sabbath,led zeppelin,kendrick lamar,kanye west,scorpions,"tyler, the creator",2pac,queen,jay-z
0,1,1,1,0,0,0,0,0,0,0
1,0,0,0,1,1,1,0,0,0,0
2,0,0,0,1,0,0,1,1,0,0
3,0,0,1,1,0,0,1,0,1,0
4,0,0,0,1,0,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...
49995,0,1,0,0,0,0,1,0,1,0
49996,1,1,1,0,0,0,0,0,1,0
49997,0,0,0,0,0,0,1,1,0,0
49998,0,0,0,1,0,0,0,0,1,1


In [51]:
norm_df.sum()

deep purple            8430
black sabbath         12087
led zeppelin          15279
kendrick lamar        18086
kanye west            12234
scorpions              4545
tyler, the creator    15370
2pac                   8547
queen                 18007
jay-z                  4450
dtype: int64

# Алгоритм Apriori

In [10]:
from itertools import combinations, permutations
from tqdm import tqdm

In [11]:
def count_rows(items: list[str]):
    condition = (norm_df[items[0]] == 1)
    for i in range(1, len(items)):
        condition &= (norm_df[items[i]] == 1)
    cnt = len(norm_df[condition])
    return cnt

In [12]:
k = 1
f = 4
results: list[list[set]] = []
while True:
    f_k: list[set] = []
    if k == 1:
        for item in unique_items:
            if norm_df.sum()[item] >= f:
                f_k.append({item})
    else:
        if k == 2:
            combs: list[set] = [x[0].union(x[1]) for x in combinations(results[0], r=2)]
        else:
            previous = results[k - 2]
            combs = []
            for i, elem in tqdm(enumerate(previous)):
                _set = set(sorted(elem)[:-1])
                if any(_set.issubset(x) for x in combs):
                    continue
                for j, elem1 in enumerate(previous):
                    if i == j:
                        continue
                    if _set.issubset(elem1) and all(any(len(set(subset).difference(x)) == 0 for x in previous) for subset in combinations(elem.union(elem1), r=k-1)):
                        combs.append(elem.union(elem1))
            
            # combs = []
            # for elem in possible:
            #     flag = True
            #     for subset in combinations(elem, r=k-1):
            #         cnt = count_rows(subset)
            #         if cnt < f:
            #             flag = False
            #             break
            #     if flag:
            #         combs.append(elem)
        
        for elem in combs:
            cnt = count_rows(list(elem))
            if cnt >= f:
                f_k.append(elem)
    
    if len(f_k) == 0:
        break
    results.append(f_k)
    k += 1

4it [00:00, ?it/s]
1it [00:00, ?it/s]


In [13]:
results

[[{'queen'},
  {'black sabbath'},
  {'deep purple'},
  {'led zeppelin'},
  {'scorpions'}],
 [{'black sabbath', 'queen'},
  {'deep purple', 'queen'},
  {'led zeppelin', 'queen'},
  {'black sabbath', 'led zeppelin'}],
 [{'black sabbath', 'led zeppelin', 'queen'}]]

In [14]:
results.pop(0)
while type(results[0]) != set:
    for elem in results[0]:
        results.append(elem)
    results.pop(0)
results

[{'black sabbath', 'queen'},
 {'deep purple', 'queen'},
 {'led zeppelin', 'queen'},
 {'black sabbath', 'led zeppelin'},
 {'black sabbath', 'led zeppelin', 'queen'}]

In [25]:
rules = pd.DataFrame(columns=['Условие', 'Следствие', 'Поддержка', 'Достоверность', 'Убеждённость', 'Лифт', 'Рычаг'])
# rules = {'Условие': [], 'Следствие': [], 'Поддержка': [], 'Достоверность': []}
for elem in results:
    for p in combinations(elem, r=len(elem)-1):
        condition = list(p)
        consequence = list(set(elem).difference(set(p)))
        support = count_rows(condition + consequence) / len(norm_df)
        confidence = count_rows(condition + consequence) / count_rows(condition)
        conviction = ((1 - count_rows(consequence) / len(norm_df)) / (1 - confidence) if confidence != 1 else np.inf)
        lift = (count_rows(consequence + condition) * len(norm_df)) / (count_rows(condition) * count_rows(consequence))
        leverage = (count_rows(consequence + condition) * len(norm_df) - count_rows(condition) * count_rows(consequence)) / len(norm_df) ** 2
        rules.loc[len(rules)] = [condition, consequence, support * 100, confidence * 100, conviction * 100, lift, leverage]
rules

,Условие,Следствие,Поддержка,Достоверность,Убеждённость,Лифт,Рычаг
0,[black sabbath],[queen],40.000000,85.714286,140.000000,1.071429,0.026667
1,[queen],[black sabbath],40.000000,50.000000,106.666667,1.071429,0.026667
2,[deep purple],[queen],33.333333,100.000000,inf,1.250000,0.066667
3,[queen],[deep purple],33.333333,41.666667,114.285714,1.250000,0.066667
4,[queen],[led zeppelin],46.666667,58.333333,96.000000,0.972222,-0.013333
5,[led zeppelin],[queen],46.666667,77.777778,90.000000,0.972222,-0.013333
6,[black sabbath],[led zeppelin],26.666667,57.142857,93.333333,0.952381,-0.013333
7,[led zeppelin],[black sabbath],26.666667,44.444444,96.000000,0.952381,-0.013333
8,"[black sabbath, queen]",[led zeppelin],26.666667,66.666667,120.000000,1.111111,0.026667
9,"[black sabbath, led zeppelin]",[queen],26.666667,100.000000,inf,1.250000,0.053333


In [26]:
rules['Поддержка'].max()

np.float64(46.666666666666664)

In [27]:
rules['Достоверность'].max()

np.float64(100.0)

In [28]:
rules[(rules['Достоверность'] >= 30)].sort_values('Достоверность')

,Условие,Следствие,Поддержка,Достоверность,Убеждённость,Лифт,Рычаг
3,[queen],[deep purple],33.333333,41.666667,114.285714,1.250000,0.066667
7,[led zeppelin],[black sabbath],26.666667,44.444444,96.000000,0.952381,-0.013333
1,[queen],[black sabbath],40.000000,50.000000,106.666667,1.071429,0.026667
6,[black sabbath],[led zeppelin],26.666667,57.142857,93.333333,0.952381,-0.013333
10,"[queen, led zeppelin]",[black sabbath],26.666667,57.142857,124.444444,1.224490,0.048889
4,[queen],[led zeppelin],46.666667,58.333333,96.000000,0.972222,-0.013333
8,"[black sabbath, queen]",[led zeppelin],26.666667,66.666667,120.000000,1.111111,0.026667
5,[led zeppelin],[queen],46.666667,77.777778,90.000000,0.972222,-0.013333
0,[black sabbath],[queen],40.000000,85.714286,140.000000,1.071429,0.026667
2,[deep purple],[queen],33.333333,100.000000,inf,1.250000,0.066667


# Алгоритм Eclat

In [29]:
eclat_dict = {'Item': [], 'Transactions': []}
min_support = 4
for item in sorted(unique_items):
    transactions = norm_df[norm_df[item] == 1].index.to_list()
    eclat_dict['Item'].append([item])
    eclat_dict['Transactions'].append(transactions)
eclat_df = pd.DataFrame(eclat_dict)
eclat_df = eclat_df[eclat_df['Transactions'].apply(lambda t: len(t) >= min_support)]
eclat_df

,Item,Transactions
0,[black sabbath],"[0, 2, 8, 10, 11, 12, 14]"
1,[deep purple],"[0, 5, 7, 8, 14]"
2,[led zeppelin],"[2, 4, 6, 7, 8, 9, 11, 12, 13]"
3,[queen],"[0, 1, 2, 3, 5, 6, 7, 8, 11, 12, 13, 14]"
4,[scorpions],"[3, 5, 10, 11]"


In [30]:
min_support = 4
start_point = 0
k = 2
while True:
    end_point = len(eclat_df)
    
    new_dict = {'Item': [], 'Transactions': []}
    for i in tqdm(range(start_point, end_point)):
        for j in range(i + 1, end_point):
            item1, item2 = eclat_df.iloc[i]['Item'], eclat_df.iloc[j]['Item']
            if k > 2 and not all(x == y for x in item1[:k - 2] for y in item2[:k - 2]):
                break

            tr1, tr2 = eclat_df.iloc[i]['Transactions'], eclat_df.iloc[j]['Transactions']
            if len(set(item1).union(set(item2))) == k:
                new_dict['Item'].append(sorted(set(item1).union(set(item2))))
                new_dict['Transactions'].append(list(set(tr1).intersection(set(tr2))))
    
    eclat_df = pd.concat([
        eclat_df,
        pd.DataFrame(new_dict)
    ], ignore_index=True)
    if len(eclat_df) == end_point:
        break
    k += 1
    start_point = end_point
    eclat_df = eclat_df[eclat_df['Transactions'].apply(lambda t: len(t) >= min_support)]

eclat_df

100%|██████████| 1/1 [00:00<?, ?it/s]


,Item,Transactions
0,[black sabbath],"[0, 2, 8, 10, 11, 12, 14]"
1,[deep purple],"[0, 5, 7, 8, 14]"
2,[led zeppelin],"[2, 4, 6, 7, 8, 9, 11, 12, 13]"
3,[queen],"[0, 1, 2, 3, 5, 6, 7, 8, 11, 12, 13, 14]"
4,[scorpions],"[3, 5, 10, 11]"
5,"[black sabbath, led zeppelin]","[8, 2, 11, 12]"
6,"[black sabbath, queen]","[0, 2, 8, 11, 12, 14]"
7,"[deep purple, queen]","[0, 5, 7, 8, 14]"
8,"[led zeppelin, queen]","[2, 6, 7, 8, 11, 12, 13]"
9,"[black sabbath, led zeppelin, queen]","[8, 2, 11, 12]"


In [31]:
eclat_df = eclat_df[eclat_df['Item'].apply(lambda x: len(x) > 1)]

In [32]:
rules = pd.DataFrame(columns=['Условие', 'Следствие', 'Поддержка', 'Достоверность', 'Убеждённость', 'Лифт', 'Рычаг'])
# rules = {'Условие': [], 'Следствие': [], 'Поддержка': [], 'Достоверность': []}
for elem in eclat_df['Item']:
    for p in combinations(elem, r=len(elem)-1):
        condition = list(p)
        consequence = list(set(elem).difference(set(p)))
        support = count_rows(condition + consequence) / len(norm_df)
        confidence = count_rows(condition + consequence) / count_rows(condition)
        conviction = ((1 - count_rows(consequence) / len(norm_df)) / (1 - confidence) if confidence != 1 else np.inf)
        lift = (count_rows(consequence + condition) * len(norm_df)) / (count_rows(condition) * count_rows(consequence))
        leverage = (count_rows(consequence + condition) * len(norm_df) - count_rows(condition) * count_rows(consequence)) / len(norm_df) ** 2
        rules.loc[len(rules)] = [condition, consequence, support * 100, confidence * 100, conviction * 100, lift, leverage]
rules

,Условие,Следствие,Поддержка,Достоверность,Убеждённость,Лифт,Рычаг
0,[black sabbath],[led zeppelin],26.666667,57.142857,93.333333,0.952381,-0.013333
1,[led zeppelin],[black sabbath],26.666667,44.444444,96.000000,0.952381,-0.013333
2,[black sabbath],[queen],40.000000,85.714286,140.000000,1.071429,0.026667
3,[queen],[black sabbath],40.000000,50.000000,106.666667,1.071429,0.026667
4,[deep purple],[queen],33.333333,100.000000,inf,1.250000,0.066667
5,[queen],[deep purple],33.333333,41.666667,114.285714,1.250000,0.066667
6,[led zeppelin],[queen],46.666667,77.777778,90.000000,0.972222,-0.013333
7,[queen],[led zeppelin],46.666667,58.333333,96.000000,0.972222,-0.013333
8,"[black sabbath, led zeppelin]",[queen],26.666667,100.000000,inf,1.250000,0.053333
9,"[black sabbath, queen]",[led zeppelin],26.666667,66.666667,120.000000,1.111111,0.026667


In [33]:
rules['Поддержка'].max()

np.float64(46.666666666666664)

In [34]:
rules['Достоверность'].max()

np.float64(100.0)

In [35]:
rules[(rules['Достоверность'] >= 30)].sort_values('Достоверность')

,Условие,Следствие,Поддержка,Достоверность,Убеждённость,Лифт,Рычаг
5,[queen],[deep purple],33.333333,41.666667,114.285714,1.250000,0.066667
1,[led zeppelin],[black sabbath],26.666667,44.444444,96.000000,0.952381,-0.013333
3,[queen],[black sabbath],40.000000,50.000000,106.666667,1.071429,0.026667
0,[black sabbath],[led zeppelin],26.666667,57.142857,93.333333,0.952381,-0.013333
10,"[led zeppelin, queen]",[black sabbath],26.666667,57.142857,124.444444,1.224490,0.048889
7,[queen],[led zeppelin],46.666667,58.333333,96.000000,0.972222,-0.013333
9,"[black sabbath, queen]",[led zeppelin],26.666667,66.666667,120.000000,1.111111,0.026667
6,[led zeppelin],[queen],46.666667,77.777778,90.000000,0.972222,-0.013333
2,[black sabbath],[queen],40.000000,85.714286,140.000000,1.071429,0.026667
4,[deep purple],[queen],33.333333,100.000000,inf,1.250000,0.066667


# Алгоритм FP-Growth

In [52]:
item_count = {artist: count_rows([artist]) for artist in unique_items}
item_count

{'deep purple': 8430,
 'black sabbath': 12087,
 'led zeppelin': 15279,
 'kendrick lamar': 18086,
 'kanye west': 12234,
 'scorpions': 4545,
 'tyler, the creator': 15370,
 '2pac': 8547,
 'queen': 18007,
 'jay-z': 4450}

In [ ]:
transaction_list: list[list[str]] = []
for trid in df['Transaction'].unique():
    transaction_list.append(df[df['Transaction'] == trid]['Item'].sort_values().to_list())
    transaction_list[-1].sort(key=lambda x: item_count[x], reverse=True)

[['led zeppelin', 'black sabbath', 'deep purple'],
 ['kendrick lamar', 'kanye west', 'scorpions'],
 ['kendrick lamar', 'tyler, the creator', '2pac'],
 ['kendrick lamar', 'queen', 'tyler, the creator', 'led zeppelin'],
 ['kendrick lamar', 'tyler, the creator', 'jay-z'],
 ['kendrick lamar', '2pac', 'scorpions'],
 ['kendrick lamar', 'kanye west', '2pac', 'scorpions'],
 ['queen', 'tyler, the creator'],
 ['kendrick lamar', 'tyler, the creator'],
 ['kanye west'],
 ['kendrick lamar', 'kanye west', 'black sabbath', 'jay-z'],
 ['kendrick lamar', 'tyler, the creator', '2pac'],
 ['queen', 'led zeppelin', 'black sabbath'],
 ['kendrick lamar', 'led zeppelin', 'deep purple'],
 ['2pac'],
 ['tyler, the creator', 'led zeppelin', 'kanye west', '2pac'],
 ['kendrick lamar', '2pac'],
 ['led zeppelin', 'kanye west', 'deep purple'],
 ['black sabbath', 'deep purple'],
 ['kendrick lamar', 'queen', 'tyler, the creator', '2pac'],
 ['kendrick lamar', 'queen', 'deep purple'],
 ['queen', 'led zeppelin'],
 ['deep pu

In [54]:
class FPTree:
    num: int = 0

    def __init__(self, value: str | None = None):
        self.value: str | None = value
        self.cnt: int = 0
        self.children: list['FPTree'] = []
        self.num: int = FPTree.num
        FPTree.num += 1
    
    def add(self, value: str):
        if value not in [x.value for x in self.children]:
            node = FPTree(value)
            node.cnt += 1
            self.children.append(node)
            return node
        else:
            index = [x.value for x in self.children].index(value)
            self.children[index].cnt += 1
            return self.children[index]
    
    def __str__(self):
        res = f'{self.num}({self.value}: {self.cnt}) -> [' + '  '.join([f'{x.num}({x.value}: {x.cnt})' for x in self.children]) + ']\n'
        for child in self.children:
            res += str(child)
        return res
    
    def find(self, value: str):
        result = dict.fromkeys(unique_items, 0)

        def dfs(node: 'FPTree'):
            cnt = 0
            for child in node.children:
                if child.value == value:
                    cnt = child.cnt
                    node.children.remove(child)
                else:
                    cnt += dfs(child)
                if node.value is not None:
                    result[node.value] += cnt
            return cnt
            
        while dfs(self) != 0:
            pass
        
        return result

In [55]:
root = FPTree()
node: FPTree = root
for transaction in transaction_list:
    for item in transaction:
        node = node.add(item)
    node = root
print(root)

0(None: 0) -> [1(led zeppelin: 4883)  4(kendrick lamar: 18086)  17(queen: 13745)  19(kanye west: 2428)  26(2pac: 1007)  27(tyler, the creator: 6110)  33(black sabbath: 1967)  37(deep purple: 950)  87(scorpions: 430)  95(jay-z: 394)]
1(led zeppelin: 4883) -> [2(black sabbath: 1443)  31(kanye west: 668)  50(deep purple: 549)  58(scorpions: 232)  62(2pac: 237)  184(jay-z: 69)]
2(black sabbath: 1443) -> [3(deep purple: 379)  93(scorpions: 151)  107(jay-z: 28)  165(2pac: 120)]
3(deep purple: 379) -> [292(scorpions: 44)  304(jay-z: 14)]
292(scorpions: 44) -> []
304(jay-z: 14) -> []
93(scorpions: 151) -> [327(jay-z: 4)]
327(jay-z: 4) -> []
107(jay-z: 28) -> []
165(2pac: 120) -> [166(scorpions: 11)  332(deep purple: 23)  377(jay-z: 5)]
166(scorpions: 11) -> []
332(deep purple: 23) -> []
377(jay-z: 5) -> []
31(kanye west: 668) -> [32(deep purple: 88)  182(black sabbath: 190)  197(scorpions: 37)  221(2pac: 105)  345(jay-z: 31)]
32(deep purple: 88) -> [278(scorpions: 14)  302(jay-z: 3)]
278(scorp

In [56]:
result = []
for item in sorted(unique_items, key=lambda x: item_count[x]):
    search_result = root.find(item)
    passed_threshold = [k for k, v in search_result.items() if v >= f]
    for i in range(1, len(passed_threshold) + 1):
        for comb in combinations(passed_threshold, r=i):
            result.append([item] + list(comb))
result

[['jay-z', 'deep purple'],
 ['jay-z', 'black sabbath'],
 ['jay-z', 'led zeppelin'],
 ['jay-z', 'kendrick lamar'],
 ['jay-z', 'kanye west'],
 ['jay-z', 'scorpions'],
 ['jay-z', 'tyler, the creator'],
 ['jay-z', '2pac'],
 ['jay-z', 'queen'],
 ['jay-z', 'deep purple', 'black sabbath'],
 ['jay-z', 'deep purple', 'led zeppelin'],
 ['jay-z', 'deep purple', 'kendrick lamar'],
 ['jay-z', 'deep purple', 'kanye west'],
 ['jay-z', 'deep purple', 'scorpions'],
 ['jay-z', 'deep purple', 'tyler, the creator'],
 ['jay-z', 'deep purple', '2pac'],
 ['jay-z', 'deep purple', 'queen'],
 ['jay-z', 'black sabbath', 'led zeppelin'],
 ['jay-z', 'black sabbath', 'kendrick lamar'],
 ['jay-z', 'black sabbath', 'kanye west'],
 ['jay-z', 'black sabbath', 'scorpions'],
 ['jay-z', 'black sabbath', 'tyler, the creator'],
 ['jay-z', 'black sabbath', '2pac'],
 ['jay-z', 'black sabbath', 'queen'],
 ['jay-z', 'led zeppelin', 'kendrick lamar'],
 ['jay-z', 'led zeppelin', 'kanye west'],
 ['jay-z', 'led zeppelin', 'scorpion

In [57]:
rules = pd.DataFrame(columns=['Условие', 'Следствие', 'Поддержка', 'Достоверность', 'Убеждённость', 'Лифт', 'Рычаг'])
# rules = {'Условие': [], 'Следствие': [], 'Поддержка': [], 'Достоверность': []}
for elem in result:
    for p in combinations(elem, r=len(elem)-1):
        condition = list(p)
        consequence = list(set(elem).difference(set(p)))
        support = count_rows(condition + consequence) / len(norm_df)
        if count_rows(condition) == 0:
            continue
        confidence = count_rows(condition + consequence) / count_rows(condition)
        conviction = ((1 - count_rows(consequence) / len(norm_df)) / (1 - confidence) if confidence != 1 else np.inf)
        lift = (count_rows(consequence + condition) * len(norm_df)) / (count_rows(condition) * count_rows(consequence))
        leverage = (count_rows(consequence + condition) * len(norm_df) - count_rows(condition) * count_rows(consequence)) / len(norm_df) ** 2
        rules.loc[len(rules)] = [condition, consequence, support * 100, confidence * 100, conviction * 100, lift, leverage]
rules

,Условие,Следствие,Поддержка,Достоверность,Убеждённость,Лифт,Рычаг
0,[jay-z],[deep purple],0.796,8.943820,91.306269,0.530476,-0.007045
1,[deep purple],[jay-z],0.796,4.721234,95.614168,0.530476,-0.007045
2,[jay-z],[black sabbath],1.194,13.415730,87.574799,0.554965,-0.009575
3,[black sabbath],[jay-z],1.194,4.939191,95.833394,0.554965,-0.009575
4,[jay-z],[led zeppelin],1.478,16.606742,83.270520,0.543450,-0.012417
...,...,...,...,...,...,...,...
2539,"[tyler, the creator, kendrick lamar]",[queen],2.208,16.030202,76.201207,0.445110,-0.027526
2540,"[tyler, the creator, queen]",[kendrick lamar],2.208,31.751510,93.522948,0.877792,-0.003074
2541,"[kendrick lamar, queen]","[tyler, the creator]",2.208,25.903332,93.472489,0.842659,-0.004123
2542,[queen],[kendrick lamar],8.524,23.668573,83.619556,0.654334,-0.045030


In [58]:
rules.max()

Условие          [tyler, the creator, queen]
Следствие               [tyler, the creator]
Поддержка                             13.774
Достоверность                      46.476064
Убеждённость                      119.251319
Лифт                                1.284863
Рычаг                               0.026547
dtype: object

In [59]:
rules['Достоверность'].max()

np.float64(46.476063829787236)

In [62]:
rules[(rules['Лифт'] >= 1)].sort_values('Достоверность')

,Условие,Следствие,Поддержка,Достоверность,Убеждённость,Лифт,Рычаг
101,"[kanye west, tyler, the creator]",[jay-z],0.818,9.064716,100.181136,1.018507,0.000149
1125,"[black sabbath, led zeppelin]",[scorpions],0.812,9.237770,100.162810,1.016256,0.000130
1122,"[deep purple, queen]",[scorpions],0.684,9.336609,100.272005,1.027130,0.000181
92,"[kendrick lamar, 2pac]",[jay-z],0.690,9.415939,100.569569,1.057971,0.000378
1087,[deep purple],[scorpions],1.624,9.632266,100.600066,1.059655,0.000914
...,...,...,...,...,...,...,...
2195,"[2pac, tyler, the creator]",[kendrick lamar],2.766,44.844358,115.723429,1.239753,0.005349
88,"[jay-z, tyler, the creator]",[kendrick lamar],1.440,45.056320,116.169868,1.245613,0.002839
1804,"[deep purple, led zeppelin]",[queen],2.764,45.415708,117.224201,1.261057,0.005722
2383,"[black sabbath, led zeppelin]",[queen],4.008,45.597270,117.615420,1.266098,0.008424
